# Load packages, types, and functions

In [1]:
using Oceananigans
using Oceananigans.Units
using JLD2
using Printf
using Oceananigans.TurbulenceClosures: TKEDissipationVerticalDiffusivity
using ParameterEstimocean: iterate!
using SingleColumnModelCalibration:
    build_ensemble_kalman_inversion,
    generate_filepath,
    parameter_sets

Precompiling Oceananigans
  ✓ Oceananigans
  1 dependency successfully precompiled in 12 seconds. 189 already precompiled.
Precompiling ParameterEstimocean
  ✓ ParameterEstimocean
  1 dependency successfully precompiled in 6 seconds. 256 already precompiled.
Precompiling SingleColumnModelCalibration
  ✓ SingleColumnModelCalibration
  1 dependency successfully precompiled in 5 seconds. 389 already precompiled.


# Parameters

Next we choose some parameters of the calibration -- the architecture to use (`CPU()` or `GPU()`), the number of ensemble members, the `stop_pseudotime` (alternatively, we can choose to do a fixed number of EKI iterations), and the time-step that the forward simulations will use.

In [2]:
architecture    = CPU()
Nensemble       = 100
simulation_Δt   = 1minute
stop_pseudotime = 1000

10000.0

# Closure and parameter set

Here we choose the closure and parameter set that we would like to calibrate.

Note that `parameter_set` must have corresponding entries in the following dictionaries, which are defined in src/parameter_sets.jl:

   * SingleColumnModelCalibration.parameter_sets
   * SingleColumnModelCalibration.dependent_parameter_sets
   * SingleColumnModelCalibration.boundary_library

In [11]:
closure         = TKEDissipationVerticalDiffusivity()
parameter_set   = "variable_stabilities"

"variable_stabilities"

New parameter sets can be defined by editing `src/parameter_sets.jl`. Here's the parameter sets available (for all closures), most of which pertain to `CATKEVerticalDiffusivity`:

In [12]:
@show SingleColumnModelCalibration.parameter_sets

LoadError: UndefVarError: `SingleColumnModelCalibration` not defined

# Data and grid parameters

We calibrate against 3 suites (12, 24, and 48-hour suites), and on three different grids (2, 4, and 8 meter vertical resolution).

In [4]:
suite_parameters = [
    (name = "12_hour_suite", stop_time=12hours),
    (name = "24_hour_suite", stop_time=24hours),
    (name = "48_hour_suite", stop_time=48hours),
]

# Grids to include
grid_parameters = [
    (size=32, z=(-256, 0)),
    (size=64, z=(-256, 0)),
    (size=128, z=(-256, 0)),
]

3-element Vector{@NamedTuple{size::Int64, z::Tuple{Int64, Int64}}}:
 (size = 32, z = (-256, 0))
 (size = 64, z = (-256, 0))
 (size = 128, z = (-256, 0))

# Filename

We use a utility to generate the filename for saving data automatically, just for convenience.

In [5]:
filename = "TKEDissipationVerticalDiffusivity_$parameter_set"
filepath = generate_filepath(; filename,
                             Nensemble,
                             suite_parameters,
                             grid_parameters,
                             stop_pseudotime,
                             Δt = simulation_Δt,
                             dir = ".")

@info "Saving data to $filepath"

[ Info: Saving data to ./TKEDissipationVerticalDiffusivity_variable_stabilities_Nens100_Δt60_τ10000_Nz32_Nz64_Nz128_12_hour_suite_24_hour_suite_48_hour_suite.jld2


# Build the `EnsembleKalmanInversion` object

We use a utility to build an `EnsembleKalmanInversion` object that represents the single column calibration problem.

In [8]:
eki = build_ensemble_kalman_inversion(closure, parameter_set;
                                      architecture,
                                      Nensemble,
                                      grid_parameters,
                                      suite_parameters,
                                      Δt = simulation_Δt)

┌ Info: Regridding synthetic observations...
│     original grid: 256×256×256 RectilinearGrid{Float64, Periodic, Periodic, Bounded} on CPU with 5×5×5 halo
└          new grid: 1×1×32 RectilinearGrid{Float64, Flat, Flat, Bounded} on CPU with 0×0×3 halo
┌ Info: Regridding synthetic observations...
│     original grid: 256×256×256 RectilinearGrid{Float64, Periodic, Periodic, Bounded} on CPU with 5×5×5 halo
└          new grid: 1×1×32 RectilinearGrid{Float64, Flat, Flat, Bounded} on CPU with 0×0×3 halo
┌ Info: Regridding synthetic observations...
│     original grid: 256×256×256 RectilinearGrid{Float64, Periodic, Periodic, Bounded} on CPU with 5×5×5 halo
└          new grid: 1×1×32 RectilinearGrid{Float64, Flat, Flat, Bounded} on CPU with 0×0×3 halo
┌ Info: Regridding synthetic observations...
│     original grid: 256×256×256 RectilinearGrid{Float64, Periodic, Periodic, Bounded} on CPU with 5×5×5 halo
└          new grid: 1×1×32 RectilinearGrid{Float64, Flat, Flat, Bounded} on CPU with 0×0

LoadError: DomainError with -0.08229321221592975:
sqrt was called with a negative real argument but will only return a complex result if called with a complex argument. Try sqrt(Complex(x)).
Stacktrace:
  [1] [0m[1mthrow_complex_domainerror[22m[0m[1m([22m[90mf[39m::[0mSymbol, [90mx[39m::[0mFloat64[0m[1m)[22m
[90m    @[39m [90mBase.Math[39m [90m./[39m[90m[4mmath.jl:33[24m[39m
  [2] [0m[1msqrt[22m
[90m    @[39m [90m./[39m[90m[4mmath.jl:686[24m[39m[90m [inlined][39m
  [3] [0m[1mminimum_stratification_number[22m
[90m    @[39m [90m~/Projects/Oceananigans.jl/src/TurbulenceClosures/turbulence_closure_implementations/TKEBasedVerticalDiffusivities/[39m[90m[4mtke_dissipation_stability_functions.jl:166[24m[39m[90m [inlined][39m
  [4] [0m[1mmomentum_stability_functionᶜᶜᶠ[22m
[90m    @[39m [90m~/Projects/Oceananigans.jl/src/TurbulenceClosures/turbulence_closure_implementations/TKEBasedVerticalDiffusivities/[39m[90m[4mtke_dissipation_stabil

# Run the calibration

Finally, we are ready to run the calibration

In [9]:
# Run the calibration
max_iterations = 10

10

In [10]:
while (eki.pseudotime < stop_pseudotime) && (eki.iteration < max_iterations)
    @time iterate!(eki)

    if eki.iteration % 5 == 0
        @show eki.iteration_summaries[end]

        rm(filepath; force=true)

        @info "Saving data to $filepath..."
        file = jldopen(filepath, "a+")
        file["resample_failure_fraction"] = resample_failure_fraction
        file["stop_pseudotime"] = stop_pseudotime
        file["iteration_summaries"] = eki.iteration_summaries
        close(file)
    end
end

LoadError: UndefVarError: `eki` not defined